## ALE single analysis

This code is largely based on the implementation provided by Enge et al. (2021), available at https://osf.io/34ry2/. We are deeply grateful for their dedication to open research.

> Enge, A., Abdel Rahman, R., & Skeide, M. A. (2021). A meta-analysis of fMRI studies of semantic cognition in children. NeuroImage, 241, 118436. https://doi.org/10.1016/j.neuroimage.2021.118436

In [1]:
# Import necessary modules
from os import makedirs, path
import os
import numpy as np
import pandas as pd
from IPython.display import display
from nibabel import save
from nilearn import image, plotting, reporting
from nimare import correct, io, meta, utils 
from scipy.stats import norm
from nimare.utils import tal2mni


# print the current working directory
# print(os.getcwd())

In [2]:
# Define function for performing a single ALE analysis with FWE correction
def run_ale(text_file, voxel_thresh, cluster_thresh, random_seed, n_iters, output_dir):

    # Set a random seed to make the results reproducible
    if random_seed:
        np.random.seed(random_seed)

    # Perform the ALE
    dset = io.convert_sleuth_to_dataset(text_file=text_file, target="ale_2mm")
    ale = meta.cbma.ALE()
    res = ale.fit(dset)

    # FWE correction for multiple comparisons
    corr = correct.FWECorrector(
        method="montecarlo", voxel_thresh=voxel_thresh, n_iters=n_iters
    )
    cres = corr.transform(result=res)

    # Save maps to the ouput directory
    prefix = path.basename(text_file).replace(".txt", "")
    res.save_maps(output_dir=output_dir, prefix=prefix)
    cres.save_maps(output_dir=output_dir, prefix=prefix)

    # Create cluster-level thresholded z and ALE maps
    img_clust_mass = cres.get_map("z_desc-mass_level-cluster_corr-FWE_method-montecarlo")
    img_clust_size = cres.get_map("z_desc-size_level-cluster_corr-FWE_method-montecarlo")
    img_z = cres.get_map("z")
    img_ale = cres.get_map("stat")
    cluster_thresh_z = norm.ppf(1 - cluster_thresh / 2)
    img_clust_mass_thresh = image.threshold_img(img=img_clust_mass, threshold=cluster_thresh_z)
    img_clust_size_thresh = image.threshold_img(img=img_clust_size, threshold=cluster_thresh_z)
    img_mask = image.math_img("np.where(img > 0, 1, 0)", img=img_clust_size_thresh)
    img_z_thresh = image.math_img("img1 * img2", img1=img_mask, img2=img_z)
    img_ale_thresh = image.math_img("img1 * img2", img1=img_mask, img2=img_ale)

    # Save thresholded maps to the output directory
    save(img=img_clust_mass_thresh, filename=output_dir + "/" + prefix + "_z_mass_level_thresh.nii.gz")
    save(img=img_clust_size_thresh, filename=output_dir + "/" + prefix + "_z_size_level_thresh.nii.gz")
    # Save ALE thresholded for tables later on
    save(img=img_z_thresh, filename=output_dir + "/" + prefix + "_z_thresh.nii.gz")
    save(img=img_ale_thresh, filename=output_dir + "/" + prefix + "_stat_size_thresh.nii.gz")

In [3]:
# Define the path to the text file containing the data

text_file = "../1_Data/AnalysisData/InputData_ALE/hyperactivation.txt"
# text_file = "../1_Data/AnalysisData/InputData_ALE/hypoactivation.txt"
# text_file = "../1_Data/AnalysisData/InputData_ALE/hyper_and_hypo.txt"

# Define output directory
output_dir="../3_Output/ALE"

In [ ]:
# Run the ALE (Activation Likelihood Estimation) analysis
run_ale(
    text_file=text_file,
    voxel_thresh=0.001,
    cluster_thresh=0.05,
    random_seed=2025,
    n_iters=10000,
    output_dir=output_dir
)

INFO:nimare.correct:Using correction method implemented in Estimator: nimare.meta.cbma.ale.ALE.correct_fwe_montecarlo.
100%|██████████| 5000/5000 [20:24<00:00,  4.08it/s]
INFO:nimare.meta.cbma.base:Using null distribution for voxel-level FWE correction.
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_33970/3490095354.py:30: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  img_clust_mass_thresh = image.threshold_img(img=img_clust_mass, threshold=cluster_thresh_z)
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_33970/3490095354.py:30: UserWarning: The given float value must not exceed 0.014037579350310981. But, you have given threshold=1.959963984540054.
  img_clust_mass_thresh = image.threshold_img(img=img_clust_mass, threshold=cluster_thresh_z)
/var/fold